In [1]:
import pandas as pd
import numpy as np 
import re

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv(r"C:\Users\user\Documents\pandas_study\bootcamp_DS\project\section2_porject_210222\winemag-data-130k-v2.csv", index_col=0)

In [3]:
df = data

In [4]:
object_columns = ['country', 'description', 'designation', 'province', 'region_1',
       'region_2', 'taster_name', 'taster_twitter_handle', 'title', 'variety',
       'winery']

for column in df[~df.isnull()][object_columns]:
    df[column] = df[column].str.replace("reserva","reserve")
    df[column] = df[column].str.replace("riserva","reserve")

In [5]:
for column in object_columns:
    df[column] = df[column].str.lower()

In [6]:
# 먼저 (로 split하고 1번째 인덱스를 region으로 저장 
df['region'] = df.title.str.split('(').str[1]

# 먼저 )로 split하고 1번째 인덱스를 region으로 저장 
df['region'] = df.region.str.split(')').str[0]

region_count = pd.DataFrame(df["region"].value_counts())
region_count["region"] = region_count["region"]/len(region_count["region"])
df = pd.merge(df, region_count, left_on="region", right_index=True)

winery_count = pd.DataFrame(df["winery"].value_counts())
winery_count["winery"] = winery_count["winery"]/len(winery_count["winery"])
df = pd.merge(df, winery_count, left_on="winery", right_index=True)

variety_count = pd.DataFrame(df["variety"].value_counts())
variety_count["variety"] = variety_count["variety"]/len(variety_count["variety"])
df = pd.merge(df, variety_count, left_on="variety", right_index=True)


designation_count = pd.DataFrame(df["designation"].value_counts())
designation_count["designation"] = designation_count["designation"]/len(designation_count["designation"])
df = pd.merge(df, designation_count, left_on="designation", right_index=True)

province_count = pd.DataFrame(df["province"].value_counts())
province_count["province"] = province_count["province"]/len(province_count["province"])
df = pd.merge(df, province_count, left_on="province", right_index=True)

country_count = pd.DataFrame(df["country"].value_counts())
country_count["country"] = country_count["country"]/len(country_count["country"])
df = pd.merge(df, country_count, left_on="country", right_index=True)

In [7]:
feature_list = ['points', 'price','winery_y', 'variety_y', 'designation_y', 'province_y',
       'region_y', 'country_y']

target = 'price'
df1 = df[feature_list]
df1 = df1.dropna()

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler 
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder

In [9]:
train, test = train_test_split(df1, train_size=0.80, test_size=0.20, random_state=42)

X_train = train.drop(columns=target)
y_train = train[target]
X_test = test.drop(columns=target)
y_test = test[target]

In [10]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20, random_state=42)

X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]

In [11]:
encoder = OneHotEncoder()
scaler = StandardScaler(with_mean=False)
X_train_encoded = encoder.fit_transform(X_train) # 학습데이터
X_train_encoded = scaler.fit_transform(X_train_encoded)
X_val_encoded = encoder.transform(X_val) # 검증데이터
X_val_encoded = scaler.fit_transform(X_val_encoded)

In [12]:
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

param = { 
    #50 ~ 100으로 실행했을 때 최적이 100옴
    # 범위 변경 : (50~100) -> (50~500)
    'n_estimators': randint(50, 500), 
    # 과적합을 방지하기 위해 max를 20으로 설정함 
    'max_depth': [5, 10, 15, 20, None], 
    # 최대 선택할 특성 수
    'max_features': uniform(0, 1), 
}

search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    param_distributions=param, 
    n_iter=5, # 데이터의 훈련횟수
    cv=3, # 교차검증 횟수
    # mae은 에러 측정용으로써 낮을 수록 좋은데
    # sklearn은 값이 클수록 좋다고 평가를 하기에 neg를 붙혀서 scoring으로 사용 
    scoring='neg_mean_absolute_error', 
    verbose=10, # 실행 과정 출력 여부
    return_train_score=True, 
    n_jobs=-1, 
    random_state=42
)


search.fit(X_train_encoded, y_train);

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:   13.3s remaining:  1.4min
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:   14.2s remaining:   39.2s
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:   15.9s remaining:   23.9s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  1.0min remaining:   54.8s
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:  4.8min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  5.0min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  5.7min finished


In [13]:
print('최적 하이퍼파라미터: ', search.best_params_)
print('CV MAE: ', -search.best_score_)
model = search.best_estimator_

최적 하이퍼파라미터:  {'max_depth': None, 'max_features': 0.6011150117432088, 'n_estimators': 201}
CV MAE:  11.818358042413374


In [14]:
model.fit(X_train, y_train);

In [15]:
print(f"최적 모델 훈련 스코어 : {model.score(X_train, y_train)}")
print(f"최적 모델 테스트 스코어 : {model.score(X_test, y_test)}")

ValueError: Number of features of the model must match the input. Model n_features is 7 and input n_features is 825 

In [ ]:
row = X_test.iloc[[20]] 
row

In [ ]:
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(row)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    features=row
)

In [ ]:
country_count

In [16]:
df["points"].mean()

88.74221283820552